In [ ]:
import pydicom
import os
import sys
import json
import numpy as np
from tqdm import tqdm
from datetime import datetime
import dicom2nifti
import dicom2nifti.settings as settings
import numpy as np
from nibabel.testing import data_path
import nibabel as nib
import imageio
from PIL import Image, ImageOps
from scipy import ndimage
import gdcm
import imageio
from matplotlib.animation import FuncAnimation, PillowWriter  
import matplotlib.animation as animation
import shutil
import matplotlib.pyplot as plt
import glob
import gc
import time
import configparser
import logging
import cv2

In [ ]:
def normalize_image(im):
    add = 0 - im.min()
    im = im + add
    im = im /(im.max() / 256)
    return im

In [7]:
def normalize_mask(im):
    final_image = np.zeros([im.shape[0],im.shape[1]],dtype=np.uint8)
    values = []
    for y in range(im.shape[0]):
        for x in range(im.shape[1]):
            final_image[y][x] = 100 * im[y][x] 
            """if im[y][x] == 0:
                continue
            elif im[y][x] == 1 or im[y][x] == 2:
                final_image[y][x] = 255
            elif im[y][x] == 3:
                final_image[y][x] = 100"""
    return final_image

In [8]:
# Change the path to your path
#folders = ["ct_scans", "infection_mask", "lung_and_infection_mask", "lung_mask"]
folders = ["masks"]

for folder in folders:
    print()
    files = os.listdir("./"+folder+"/")
    files.sort()
    for file in files:
        if file.endswith("nii"):
            path = "./"+folder+"/" + file
            my_img  = nib.load(path)
            nii_data = my_img.get_fdata()
            nii_aff  = my_img.affine
            nii_hdr  = my_img.header
            print(nii_aff ,'\n',nii_hdr)
            print(nii_data.shape)
            if(len(nii_data.shape)==3):
                for slice_Number in range(nii_data.shape[2]):
                    if "mask" in folder:
                        im = normalize_mask(nii_data[:,:,slice_Number])
                    else:
                        im = normalize_image(nii_data[:,:,slice_Number])
                    #im = nii_data[:,:,slice_Number]
                    cv2.imwrite(path + "_" + str(slice_Number) + ".png", im)
            if(len(nii_data.shape)==4):
                for frame in range(nii_data.shape[3]):
                    for slice_Number in range(nii_data.shape[2]):
                        if "mask" in folder:
                            im = normalize_mask(nii_data[:,:,slice_Number,frame])
                        else:
                            im = normalize_image(nii_data[:,:,slice_Number,frame])
                        #im = nii_data[:,:,slice_Number,frame]
                        cv2.imwrite(path + "_" + str(slice_Number) + ".png", im)


[[-9.22999978e-01 -0.00000000e+00  0.00000000e+00  2.22194000e+02]
 [ 0.00000000e+00  9.22999978e-01  0.00000000e+00 -2.35786484e+02]
 [ 0.00000000e+00  0.00000000e+00  8.00000000e+00 -9.61700012e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]] 
 <class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 512 512  38   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [-1.     0.923  0.923  8.     0.     0.     0.     0.   ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0